In [1]:
import csv
from datetime import timedelta
import gurobipy as gp 
from gurobipy import GRB
import networkx as nx
import random
import matplotlib.pyplot as plt
import inspect
from collections import defaultdict

from models import simple_mpc, mpc_duration_constr, lazy, column_generation
from helper import Service, hhmm2mins, mins2hhmm, fetch_data, draw_graph_with_edges, node_legal, no_overlap, create_duty_graph, extract_nodes, generate_paths, roster_statistics, solution_verify, solve_RMLP, new_duty_with_bellman_ford

In [2]:
import importlib
import models
import helper

importlib.reload(models)
importlib.reload(helper)

from models import simple_mpc, mpc_duration_constr, lazy, column_generation
from helper import Service, hhmm2mins, mins2hhmm, fetch_data, draw_graph_with_edges, node_legal, no_overlap, create_duty_graph, extract_nodes, generate_paths, roster_statistics, solution_verify, solve_RMLP, new_duty_with_bellman_ford

In [3]:
services, service_dict = fetch_data('./StepBackServices.csv')
graph = create_duty_graph(services)

In [4]:
init_duties, duty_count = mpc_duration_constr(graph, service_dict, time_limit = 30, show_logs = True, show_duties = False, show_roster_stats = True)
init_duties2 = []
for duty in init_duties:
    init_duties2.append([-2] + duty + [-1])
print(init_duties2[:3])

Set parameter Username
Set parameter LicenseID to value 2626143
Academic license - for non-commercial use only - expires 2026-02-21
Set parameter TimeLimit to value 30
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 104364 rows, 68020 columns and 335422 nonzeros
Model fingerprint: 0x4522e447
Variable types: 34478 continuous, 33542 integer (33542 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 4053 rows and 1231 columns
Presolve time: 1.91s
Presolved: 100311 rows, 66789 columns, 375048 nonzeros
Variable types: 33433 continuous, 33356 integer (33356 binary)
Found heuristic solution: objective 899.0000000
Deterministic concurrent LP optimiz

In [5]:
column_generation(method=1, graph = graph, services = services, init_duties= init_duties2, num_iter = 11)

Optimal solution found
Positive Duties, > 0.5:  160
Number of 1s is: 160
Objective Value: 160.0
[-2, 336, 344, 32, 71, 113, 776, 178, 456, 249, 842, 857, 866, -1] -12.0
Optimal solution found
Positive Duties, > 0.5:  160
Number of 1s is: 160
Objective Value: 160.0
[-2, 336, 344, 32, 71, 109, 776, 178, 456, 249, 842, 857, 866, -1] -12.0
Optimal solution found
Positive Duties, > 0.5:  160
Number of 1s is: 160
Objective Value: 160.0
[-2, 336, 344, 32, 71, 109, 776, 178, 456, 249, 842, 857, 866, -1] -12.0
Optimal solution found
Positive Duties, > 0.5:  160
Number of 1s is: 160
Objective Value: 160.0
[-2, 336, 344, 32, 71, 109, 776, 178, 456, 249, 842, 857, 866, -1] -12.0
Optimal solution found
Positive Duties, > 0.5:  160
Number of 1s is: 160
Objective Value: 160.0
[-2, 336, 344, 32, 71, 109, 776, 178, 456, 249, 842, 857, 866, -1] -12.0
Optimal solution found
Positive Duties, > 0.5:  160
Number of 1s is: 160
Objective Value: 160.0
[-2, 336, 344, 32, 71, 109, 776, 178, 456, 249, 842, 857, 8

In [6]:
init_duties3 = []
for duty in init_duties2:
    init_duties3.append([-2] + duty + [-1])
print(init_duties3[:3])

[[-2, -2, 336, 28, 67, -1, -1], [-2, -2, 872, 697, 14, 367, -1, -1], [-2, -2, 873, 347, 38, 79, 399, -1, -1]]


In [7]:
def generate_new_column(graph, service_dict, current_duties, dual_values, method = "topological sort"):
    new_duties = []
    reduced_costs = []
    # Update edge weights based on dual values
    for u, v in graph.edges():
        graph[u][v]['weight'] = int(dual_values[u])* service_dict[u] # Adjust edge weight by dual value